# Defining a station

In this example we will shortly discuss the use of a station with instruments. A station bounds instruments and has functionality to control and store the settings of the devices. We will start from a simulated station (the virtual dot array) that has a special function `initialize` to directly create the station object with simulated instruments:

In [2]:
from qtt.simulation import virtual_dot_array
station = virtual_dot_array.initialize()

c:\develop\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


initialize: create virtualdot
initialized virtual dot system (2 dots)


The simulated station contain the instuments; `gates`, `keithley1`, `keithley3`, `keithley4`, `ivvi1`, `ivvi2`, `vawg` and a `digitizer`. In general, every Instrument of a station have Parameter items, which are gettable or settable. For example, using the simulated station we can get the keithley and P1 gate amplitude:

In [3]:
gates = station.gates
print(gates)
print(gates.P1.get())

gates.P1.set(1.5)
print(gates.P1.get())

keithley1 = station.keithley1
print(keithley1)
print(keithley1.amplitude.get())

gates: gates (16 gates)
0.3166101322411219
1.5
<VirtualMeter: keithley1>
2.9964151248162425


Default workflow after connecting the instruments (the `initialize` function) is to use the station with the measurement functions. The measurements make use of the instruments in the station. To create a station, you need to define you own `initialize` function. An example `initialize` function is shown below. For your own setup to have to modify the code with the correct instruments and settings.

<pre>from qtt.instrument_drivers.gates import VirtualDAC

# the gate_map defines the relation between names gates and the DAC channels of physical instruments
gate_map = {'P0': (0,1), 'P1', (0,2), 'P3', (0,3), 'B0', (1,0), 'B1': (1,1), 'B2': (1,2), 'B3': (1,3)}

def initialize():
    
    logging.info('my station: initialize')
    
    # Loading IVVI
    logging.info('my station: load IVVI driver')
    ivvi1 = IVVI.IVVI(name='ivvi1', address='COM5', numdacs=16,)
    ivvi2 = IVVI.IVVI(name='ivvi2', address='COM6', numdacs=16,)
    
    # Loading Lockins
    logging.info('my station: load lockin driver')
    lockin1 = SR830.SR830(name='lockin1', address='GPIB1::6::INSTR')
    lockin1.output_interface('GPIB')

    # Loading digitizer
    logging.info('my station: load digitizer driver')
    digitizer = M4i.M4i(name='digitizer')

    logging.info('my station: all drivers have been loaded')
        
    # Create virtual instruments
    gates = VirtualDAC(name='gates', gate_map=gate_map, instruments=[ivvi1, ivvi2])

    #Creating the experimental station
    station = qcodes.Station(ivvi1, ivvi2,, lockin1, digitizer, gates)
    
    logging.info('my station: initialization done')   
    return station
</pre>    

A view of all the settings of a instrument or station can be shown using the `snapshot` function. The measurements use `snapshot` to collect the settings, which are stored together with the measurement results.

In [4]:
print(keithley1.snapshot())
print(station.snapshot())

{'functions': {}, 'submodules': {}, '__class__': 'qtt.instrument_drivers.virtual_instruments.VirtualMeter', 'parameters': {'IDN': {'value': None, 'ts': None, 'raw_value': None, '__class__': 'qcodes.instrument.parameter.Parameter', 'full_name': 'keithley1_IDN', 'unit': '', 'instrument': 'qtt.instrument_drivers.virtual_instruments.VirtualMeter', 'instrument_name': 'keithley1', 'label': 'IDN', 'inter_delay': 0, 'name': 'IDN', 'vals': '<Anything>', 'post_delay': 0}, 'amplitude': {'value': 2.9964151248162425, 'ts': '2018-09-16 21:34:35', 'raw_value': 2.9964151248162425, '__class__': 'qcodes.instrument.parameter.Parameter', 'full_name': 'keithley1_amplitude', 'unit': 'a.u.', 'instrument': 'qtt.instrument_drivers.virtual_instruments.VirtualMeter', 'instrument_name': 'keithley1', 'label': 'keithley1 amplitude', 'inter_delay': 0, 'name': 'amplitude', 'post_delay': 0}, 'readnext': {'value': None, 'ts': None, 'raw_value': None, '__class__': 'qcodes.instrument.parameter.Parameter', 'full_name': 'k

After all experiments are over, all devices need to be disconnected. The station has a `close` function to stop and clean up all the instrument resources:

In [35]:
virtual_dot_array.close()

close gates: gates (16 gates)
close <VirtualMeter: keithley1>
close <VirtualMeter: keithley3>
close <VirtualMeter: keithley4>
close VirtualIVVI: ivvi1
close VirtualIVVI: ivvi2
close <SimulationAWG: vawg>
close <SimulationDigitizer: sdigitizer>
